In [ ]:
try:
    # settings colab:
    import google.colab
    
    ! mkdir -p ../Data
    # los que usan colab deben modificar el token de esta url:
    ! wget -O ../Data/melb_data.csv "https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_05_Limpieza_de_datos/Data/melb_data.csv?token=AA4GFHIDAH5VEX2N7ZFSVYK6VHD3I"
    
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Funciones Lambda. Apply. Applymap.


<a id="section_toc"></a> 
## Tabla de Contenidos

[Funciones Lambda](#section_intro)

$\hspace{.5cm}$[Sintaxis](#section_sintaxis)

[Métodos apply applymap map](#section_apply)


<a id="section_lambda"></a> 
## Funciones Lambda


[volver a TOC](#section_toc)


Las funciones lambda también se denominan funciones anónimas. Una función anónima es una función definida sin un nombre. Como sabemos, para definir una función normal en Python, usamos la palabra clave `def`. Pero en el caso de funciones anónimas, usamos la palabra clave `lambda` para definirlas.

Las funciones lambda son especialmente útiles en análisis de datos porque, como veremos, hay muchos casos donde las funciones de transformación de datos tomarán **funciones como argumentos**. En general, es más claro pasar una función lambda en lugar de escribir una declaración de función completa, o de asignar una expresión lambda a una variable local.

<a id="section_sintaxis"></a> 
### Sintaxis


[volver a TOC](#section_toc)


`lambda arguments: expression`

Ejemplos:

1) Definimos una función lambda y la asignamos a la variable add. En la siguiente linea la ejecutamos e imprimimos el resultado.

In [ ]:
add = lambda a: a + a
print(add(20))

2) Definimos otra función lambda y la guardamos en la variable add, la ejecutamos e imprimimos el resultado.

In [ ]:
add = lambda a, b, c: a + b + c
print(add(10, 11, 12))

3) Rescribimos como función lambda la función `short_function`

In [ ]:
def short_function(x):
    return x * 2

equiv_anon = lambda x: x * 2

<div id="caja1" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/haciendo_foco.png" style="align:left"/> </div>
  <div style="float:left;width: 85%;"><label>
En cada unos de los ejemplos de funciones lambda:<br/>
¿Cuáles son los argumentos?
<br/>
¿Cuál es la expresión?
</label></div>
</div>



---

<a id="section_apply"></a> 
##  apply, applymap, map

[volver a TOC](#section_toc)


Pandas cuenta con un conjunto de métodos que permiten operar eficientemente sobre los elementos de un objeto DataFrame o Series.
Para aplicar la lógica deseada, podemos optar tanto por definir funciones con nombre como por utilizar expresiones lambda que luego no pueden reutilizarse.
    
1)  pd.DataFrame.apply: Opera sobre filas o columnas completas de una instancia de DataFrame

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html
    
2)  pd.DataFrame.applymap: Opera sobre cada uno de los elementos de una instancia de DataFrame

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.applymap.html
    
3)  pd.Series.apply: Opera sobre cada uno de los elementos de la instancia de Series. 
    
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.apply.html
    
4)  pd.Series.map: Opera sobre cada uno de los elementos de la instancia de Series, muy similar a Series.apply. 
    
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html

La diferencia entre pd.Series.map y pd.Series.apply es que la segunda puede generar un Dataframe a partir de la serie, mientras que la primera si recibiera una serie como return de la función crearía una serie de series.

El método apply de pandas permite realizar operaciones vectorizadas sobre los objetos DataFrame tanto fila por fila como columna por columna.

Vamos a ver ahora algunos ejemplos de uso sobre el dataset de propiedades de Melbourne, que usamos en la práctica de Limpieza de Datos

In [ ]:
import pandas as pd

# local
data_location = "../Data/melb_data.csv"
# colab
# data_location = ""

data = pd.read_csv(data_location)

data.head(3)


## Apply sobre filas de un objeto DataFrame

Vamos a calcular el precio por metro cuadrado de cada uno de los registros del dataset.

Para eso, vamos a dividir el valor del campo Price por el valor del campo Landsize para cada registro.

Escribimos esa cuenta como una función Lambda, que pasaremos como primer argumento de apply.

En la documentación, para el argumento axis vemos:

    0 or ‘index’: apply function to each column.

    1 or ‘columns’: apply function to each row.

Por lo tanto, en el segundo argumento el valor de de axis es 1 porque queremos aplicar la función a cada fila.

Como tenemos que hacer una división, debemos verificar que el denominador no sea cero. Para eso vamos a escribir una expresión condicional en una linea con esta sintaxis:

`value_when_true if condition else value_when_false`

En este caso:

`0 if x['Landsize'] == 0 else x['Price'] / x['Landsize']`


In [ ]:
precio_m2 = data.apply(lambda x: 0 if x['Landsize'] == 0 else x['Price'] / x['Landsize'], axis=1)
print(type(precio_m2))
print("Cantidad de elementos en precio_m2: " + str(len(precio_m2)))
print("Cantidad de filas en data: " + str(data.shape[0]))
print(precio_m2.head(3))

Vemos que precio_m2 es una instancia de Series y que tiene la misma cantidad de elementos que la cantidad de filas en data

## Apply sobre columnas de un objeto DataFrame

Ahora vamos a calcular el promedio de cada una de las columnas con valores numéricos.

Para determinar si una columna es de tipo numérico vamos a usar `np.issubdtype(..., np.number)` que devuelve True si col_type hereda de np.number, y False si no. Y vamos a armar una lista por comprensión con los nombres de las columnas de tipo numérico:

`numeric_columns_mask = [np.issubdtype(data[col].dtypes, np.number) for col in data.columns]
numeric_columns_names = data.columns[numeric_columns_mask]`


Para eso definiremos una función lambda que aplique `mean` sobre cada columna cuyo nombre esté en la lista que construimos en el paso anterior.

Como queremos aplicar esta función sobre cada una de las columnas del DataFrame, el valor de axis que usamos es 0.


In [ ]:
import numpy as np

numeric_columns_mask = [np.issubdtype(data[col].dtypes, np.number) for col in data.columns]
numeric_columns_names = data.columns[numeric_columns_mask]
print(numeric_columns_names)

means = data.apply(lambda x: x.mean() if x.name in numeric_columns_names else np.NaN, axis=0)
print(means)
print(type(means))
print(len(means) == data.shape[1])

Vemos que `means` es una instancia de Series y que tiene la misma cantidad de elementos que la cantidad de columnas en data

## Applymap sobre un objeto DataFrame

Vamos a escribir ahora todos los valores de los elementos del DataFrame en minúscula.

Si es el valor es numérico, no lo cambiamos.


In [ ]:
data_lower = data.applymap(lambda x: x if np.isreal(x) else str(x).lower())
# comparo los tipos de datos antes y después de pasar a minúsculas:
print(data_lower.dtypes == data.dtypes)
print(data_lower.head(3))

Vemos que los tipos de datos de las columnas del DataFrame no se modificaron, y que todas las cadenas de caracteres quedaron en minúscula.

## Apply sobre un objeto Series


Construimos una instancia de Series con los valores de la columna Price.

Vamos a aplicar un descuento del 10% en el precio, multiplicando cada uno de los valores de este objeto Series por 0.9

In [ ]:
price_serie = data.Price
print(type(price_serie))

price_discount = price_serie.apply(lambda x: x * 0.9)
price_discount

## Map sobre un objeto Series

Repetimos el ejemplo anterior con map.

In [ ]:
price_serie = data.Price
print(type(price_serie))

price_discount = price_serie.map(lambda x: x * 0.9)
price_discount

## Usando apply con funciones no anónimas

En todos los casos anteriores usamos funciones anónimas porque se ejecutan de modo más eficiente que las definidas con `def`. Sin embargo, la sintaxis de apply, applymap y map es la misma cualquiera sea la forma de difinir la función (`lambda` o `def`).

Como ejemplo, vamos a rescribir el ejercicio en el que calculamos el promedio de cada una de las columnas de data con valores numéricos, pero con una función definida con "def" (no anónima).

In [ ]:
def promedio_numerica(columna):    
    columna_numeric = pd.to_numeric(columna, errors='coerce') 
    # si no puede convertirlo va a asignar NaN a todos los elementos de la columna
    result = columna_numeric.mean()
    return result

means = data.apply(promedio_numerica, axis=0)
print(means)
print(type(means))
print(len(means) == data.shape[1])

**Observación:**

Es importante destacar que los métodos que vimos en esta práctica, apply applymap y map, son vectorizados y por lo tanto se ejecutan de forma muy eficiente. Por eso siempre vamos a preferirlos en reemplazo de los ciclos for o while.

---

## Ejercicio

Construyamos una columna nueva (de nombre postcode_suburb) en el DataFrame data, que sea la concatenación del campo Postcode, seguido de "-", seguido del valor del campo Suburb.

Para esto usemos apply sobre data, y una función lambda.

